In [4]:
import nltk
import numpy as np
import pandas as pd
import os
import glob
import random

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.special import gammaln


In [7]:
big_frame.shape

(314991, 3)

In [17]:

DIR = r'data_folder/wordcounts'
allfiles = glob.glob(os.path.join(DIR,"*.CSV"))

p = .5
rand_sample = [ allfiles[i] for i in sorted(random.sample(xrange(len(allfiles)), int(p * len(allfiles)))) ]
rand_sample
    
np_array_list = []
for file_ in rand_sample:
    df = pd.read_csv(file_,index_col=None, header=0)
    df['source'] = file_
    np_array_list.append(df.as_matrix())
    
comb_np_array = np.vstack(np_array_list)
big_frame = pd.DataFrame(comb_np_array)
big_frame.columns = ['words','count','source']
big_test = big_frame.head(n=10000)
big_test = big_test.pivot(index = 'source',columns = 'words', values = 'count')
big_test = big_test.fillna(value = 0)

In [ ]:
# There are some nonsense words in here, can we use some sort of dictionary to sweep them out?
''' 
My best guess as to why this is happening is:
1. As JSTOR parses the data things like equations and tables become jibberish.
2. Maybe we are using the wrong character format? Maybe it's unicode?
3. Aliens
'''
big_frame.head()


In [3]:
# We could do a couple of shady things like this to help remove those
big_frame = big_frame.loc[:, (big_frame.sum(axis = 0) > 2)]
big_frame = big_frame.loc[:, (big_frame.sum(axis = 0) < 20)]
big_frame.head()



words,nan,aac,aacm,aalen,aam,aamse,aaron,aban,abandon,abandoned,...,zu,zucker,zurich,zv,zw,zwet,zx,zygmund,zyskind,zz
source,,,,,,,,,,,,,,,,,,,,,
data_folder/wordcounts/wordcounts_10.2307_2276708.CSV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
data_folder/wordcounts/wordcounts_10.2307_2276742.CSV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
data_folder/wordcounts/wordcounts_10.2307_2276818.CSV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
data_folder/wordcounts/wordcounts_10.2307_2276856.CSV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
data_folder/wordcounts/wordcounts_10.2307_2276867.CSV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
def log_multinomial_beta(alpha):
    '''
    Function to take the logarithm of the multinomial beta function
    '''
    return np.sum(gammaln(alpha)) - gammaln(np.sum(alpha))

testAlpha = pd.DataFrame(random.sample(range(1,100),10),index = xrange(10), columns = xrange(1))
log_multinomial_beta(testAlpha)

0   -1057.724457
dtype: float64

In [31]:
big_frame.shape


(500, 10124)

In [28]:
#Run this, but not the one that looks the same below it
M, N = big_frame.shape

# Dimensions
# M: Number of documents
# N: Number of words
# ntopics: Number of topics

alpha = .3
beta = .3
burn_in = 10
max_iter = 100
ntopics = 6

NWZ = np.zeros((N,ntopics), dtype=np.float64) + beta
NZM = np.zeros((ntopics,M), dtype = np.float64) + alpha
NZ = NWZ.sum(axis=0)
Z = np.zeros((M,N), dtype = np.float64)
Phi = np.zeros((N,ntopics), dtype = np.float64)
Theta = np.zeros((N,ntopics), dtype = np.float64)
topicdraw = np.ones((1,ntopics), dtype = np.float64) / ntopics
read_out_Phi = np.zeros((N,ntopics), dtype = np.float64)
read_out_Theta = np.zeros((ntopics,M))
read_out_sampling_num = 0
logPw_z = np.zeros(max_iter, dtype = np.float64)
betaVec = np.ones(ntopics, dtype = np.float64) * beta
sampling_lag = 10

Z = np.where(np.random.multinomial(1,[1./ntopics]*ntopics,size = M*N )==1)[1]
Z_index = Z.reshape(M*N)
Z = Z.reshape(M,N)
big_frame_index = big_frame.stack()



In [29]:
# I keep getting memory errors, so had to use a loop to iterate
# is there some way to get around this?
for m in xrange(M):
    NZM[Z_index,m] += 1
    
NWZ[big_frame_index,Z_index] +=1
NZ[Z_index] += 1

In [6]:
NZM[Z_index,:] += 1    
NWZ[big_frame_index,Z_index] +=1
NZ[Z_index] += 1

MemoryError: 

In [ ]:
NZM

In [ ]:

# This is the old one, do not run this
M, N = big_frame.shape

# Dimensions
# M: Number of documents
# N: Number of words
# ntopics: Number of topics

# For now, try to set not small amount for alpha and beta
# else negative numbers can pop up elsewhere due to rounding error
alpha = .3
beta = .3
burn_in = 10
max_iter = 100
ntopics = 6

NWZ = np.zeros((m,ntopics), dtype=np.float64) + beta
NZM = np.zeros((ntopics,M), dtype = np.float64) + alpha
NZ = NWZ.sum(axis=0)
Z = np.zeros((M,N), dtype = np.float64)
Phi = np.zeros((M,N), dtype = np.float64)
Theta = np.zeros((N,ntopics), dtype = np.float64)
topicdraw = np.ones((1,ntopics), dtype = np.float64) / ntopics
read_out_Phi = np.zeros((N,ntopics), dtype = np.float64)
read_out_Theta = np.zeros((ntopics,M))
read_out_sampling_num = 0
logPw_z = np.zeros(max_iter, dtype = np.float64)
betaVec = np.ones(ntopics, dtype = np.float64) * beta
sampling_lag = 10


Z = np.where(np.random.multinomial(1,[1./ntopics]*ntopics,size = M*N )==1)[1]
Z = pd.DataFrame(Z.reshape(M,N))

# Draw the initial starting points
for m in xrange(M):
    for n in xrange(N):
        NZM[Z[m,n],m] = NZM[Z[m,n],m] + 1
        NWZ[big_frame[m,n],Z[m,n]] = NWZ[big_frame[m,n],Z[m,n]] + 1
        NZ[Z[m,n]] = NZ[Z[m,n]] + 1

In [ ]:
NZM

In [ ]:
# Trying to vectorize this

for iteration in xrange(maxa_iter):
    # I think we need to do it over each document
    for m in xrange(M):
        NWZ[Z_index,:] -= 1
        NWZ[big_frame_index,Z_index] -=1
        NZ[Z_index] -= 1
        

In [ ]:
p = np.zeros(ntopics, dtype = np.float64)
for k in xrange(ntopics):
    p[k] = NWZ[big_frame.iloc[m,n],k]/NZ[k] * NZM[k,m]

In [17]:
Z

array([[5, 5, 3, ..., 4, 0, 0],
       [0, 4, 0, ..., 4, 2, 0],
       [0, 3, 1, ..., 4, 4, 1],
       ..., 
       [2, 3, 0, ..., 0, 1, 2],
       [4, 3, 4, ..., 3, 2, 5],
       [0, 3, 3, ..., 1, 0, 1]])

In [30]:
Z.setflags(write=True)

for iteration in xrange(max_iter):
        for m in xrange(M):
            for n in xrange(N):
                #NZM[Z[m,n],m] -= 1
                #NWZ[big_frame.iloc[m,n],Z[m,n]] -= 1
                #NZ[Z[m,n]] -= 1
                p = np.zeros(ntopics, dtype = np.float64)
                for k in xrange(ntopics):
                    p[k] = NWZ[big_frame.iloc[m,n],k]/NZ[k] * NZM[k,m]
                
                p = p / np.sum(p)
                Z[m,n] = np.where(np.random.multinomial(1,p,size = 1 ) == 1)[1]
                NZM[Z[m,n],m] += 1
                NWZ[big_frame.iloc[m,n],Z[m,n]] += 1
                NZ[Z[m,n]] += 1
        
        
        for ZZ in xrange(ntopics):
            logPw_z[iteration] += log_multinomial_beta(NWZ[:,ZZ]) - log_multinomial_beta(betaVec)
        
        if iteration % sampling_lag == 0 or iteration % sampling_lag == 1:
            if iteration >= burn_in:
                read_out_sampling_num = read_out_sampling_num + 1
                for k in xrange(ntopics):
                    read_out_Phi[:,k] += NWZ[:,k] / NZ[k]
                
                for m in xrange(M):
                    read_out_Theta[:,m] +=  NZM[:,m]/sum(NZM[:,m])
                
Phi = read_out_Phi / read_out_sampling_num
Theta = read_out_Theta / read_out_sampling_num
        
      
        
        
        

In [27]:
read_out_Phi[:,k].shape

(10124,)

In [25]:
NWZ[:,k].shape

(500,)

In [ ]:
import matplotlib.pyplot as plt


plt.plot(logPw_z.transpose())
plt.show()